In [ ]:
'''
    Title         : Streak Analysis
    Author        : Aman Singh Thakur
    Last Edited   : 08 August, 2019
'''

from pyawake.Searching import searching
from pyawake.Visualizing import visualizing

In [ ]:
streak_dataset_cuts = "/AwakeEventData/TT41.BCTF.412340/Acquisition/totalIntensityPreferred < 100.0,/AwakeEventData/TT41.BCTF.412340/Acquisition/totalIntensityPreferred > 1.0"
#dataset_image = search_ptr.search("streak xmpp", "", "2018-05-26 00:00:00", "2018-05-26 23:59:00", streak_dataset_cuts)
dataset = searching.load_column("streak xmpp", "", "2018-05-01 00:00:00", "2018-05-31 23:59:59", streak_dataset_cuts)
#list_of_datetimes, values_exposure_time, values_time_range, values_mcp_gain, values_info, values_trig_level, list_h5_files = zip(*dataset)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import os, sys
from scipy.fftpack import fft, ifft

os.environ['AAT'] = '/eos/user/a/amthakur/SWAN_projects/AWAKE_ANALYSIS_TOOLS/'
sys.path.append(os.environ['AAT']+'ntupling/')
sys.path.append(os.environ['AAT']+'utilities/')
sys.path.append(os.environ['AAT']+'analyses/')

import frame_analysis as fa
from custom_cmap import custom_cmap

%matplotlib notebook
cm = custom_cmap()

dir_file_path = '/eos/user/a/amthakur/PNGFiles/Streak_20180526_Good_Data2/' 
list_of_files = os.listdir(dir_file_path)
val = []

ImageHeight = 512
ImageWidth = 672

def showImageTimeRange(list_of_datetimes, values,  title, xlabel, ylabel):
    values_float = []
    for val in values:
        if(val[-2:] == "ns"):
            values_float.append(float(val[:-2])*1000)
        else:
            values_float.append(float(val[:-2]))
    values_float = [float(val) for val in values_float]
    dates = matplotlib.dates.date2num(list_of_datetimes)
    fig, ax = plt.subplots()
    ax.set_title(title)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.plot_date(dates, values_float, marker='o', linestyle='-')
    plt.tight_layout()
    fig.autofmt_xdate()
    plt.show()

def showOther(list_of_datetimes, values, title, xlabel, ylabel):
    dates = matplotlib.dates.date2num(list_of_datetimes)
    fig, ax = plt.subplots()
    ax.set_title(title)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.plot_date(dates, values, marker='o', linestyle='-')
    plt.tight_layout()
    fig.autofmt_xdate()
    plt.show()
    
def filter_data(values_info):
    list_values_slit_width = []
    list_values_shutter = []
    list_values_mode = []
    for value in values_info:
        val_arr = value.split(",")
        for val in val_arr:
            if(val.find("slit")!=-1 and val.find("width")!=-1):
                width_arr = val.split("\"")
                list_values_slit_width.append(float(width_arr[1]))
            if(val.find("shutter")!=-1):
                shutter_arr = val.split("\"")
                if(shutter_arr[1]=="open"):
                    list_values_shutter.append(True)
                else:
                    list_values_shutter.append(False)
            if(val.find("operate")!=-1):
                list_values_mode.append(True)
            if(val.find("focus")!=-1 and val.find("time")==-1 and val.find("scaling")==-1):
                list_values_mode.append(False)
    return list_values_slit_width, list_values_shutter, list_values_mode

def filter_data_property(dataset, list_values_shutter, list_values_mode, list_values_slit_width):
    dataset_final = []
    for i in range(0, len(dataset)):
        if(list_values_shutter[i] == True and list_values_mode[i] == True):
            dataset_final.append(dataset[i][0:4]+dataset[i][5:8]+[list_values_slit_width[i]])
    return dataset_final
    
def filter_data_shortlist(dataset_final):
    dataset_shortlisted = []
    for i in range(0, len(dataset_final)):
        if(dataset_final[i][1]=="100 ms" and dataset_final[i][2]=="200 ps" and str(dataset_final[i][6])=="20.0"):
            dataset_shortlisted.append(dataset_final[i])
    return dataset_shortlisted

def write_values(dataset_shortlisted):
    for i in range(0, len(dataset_shortlisted)):
        file_arr = dataset_shortlisted[i][5].split("/")
        file_arr_name = file_arr[len(file_arr)-1].split(".")
        print(i, file_arr_name[0])
        if not os.path.exists(dir_file_path+file_arr_name[0]+".txt"):
            h5File = h5py.File(dataset_shortlisted[i][5], 'r')
            ImageData = list(h5File["/AwakeEventData/XMPP-STREAK/StreakImage/streakImageTimeValues"])
#             ImageData = list(h5File["/AwakeEventData/XMPP-STREAK/StreakImage/streakImageData"])
#             ImageHeight = 512
#             ImageWidth = 672
#             ImageData = np.reshape(ImageData, (ImageHeight, ImageWidth))
#             ImageData = medfilt(ImageData)
#             ImageData = np.array(ImageData)
            np.savetxt(dir_file_path+file_arr_name[0], ImageData)

def displayMovieStreak(list_of_files):
    for i in range(0, len(list_of_files)):
        image = list_of_files[i]
        ImageData = np.loadtxt(dir_file_path+image)
        width_axis = np.linspace(-7.5,7.5,ImageWidth)
        stk_ana = fa.FrameAna(ImageData,width_axis,ImageHeight)
        stk_ana.median_filter = True
        stk_ana.roi = [-4, 4, ImageWidth, ImageHeight]
        fig = plt.figure(1)
        im = plt.imshow(stk_ana.frame, extent=(stk_ana.roi[0], stk_ana.roi[1], stk_ana.roi[3], stk_ana.roi[2]), aspect='auto',cmap = cm)
        plt.colorbar(im)
        plt.title('Streak Analysis 2018/05 (Good Data)', fontsize=8)
        plt.xlabel('X [mm]',fontsize=14)
        plt.ylabel('T [ps]',fontsize=14)
        fig.canvas.draw()
        plt.clf()

    fig = plt.figure(1)
    im = plt.imshow(stk_ana.frame, 
                            extent=(stk_ana.roi[0], stk_ana.roi[1], stk_ana.roi[3], stk_ana.roi[2]), aspect='auto',cmap = cm)
    plt.colorbar(im)
    plt.title('Streak Analysis 2018/05 (Good Data)', fontsize=8)
    plt.xlabel('X [mm]',fontsize=14)
    plt.ylabel('T [ps]',fontsize=14)
    fig.canvas.draw()
    
def performFFT(list_of_files):
    fast_forward_transform_values = []
    for i in range(0, len(list_of_files)):
        if(i%10 ==0):
            print(i)
        image = list_of_files[i]
        ImageData = np.loadtxt(dir_file_path+image)
        y = fft(ImageData)
        yinv = ifft(y)
        fast_forward_transform_values.append(np.sum(yinv))
    return fast_forward_transform_values
        
def plot_image(list_of_datetimes, title, xlabel, ylabel, values):
    dates = matplotlib.dates.date2num(list_of_datetimes)
    fig, ax = plt.subplots()
    ax.set_title(title)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.plot_date(dates, values, marker='o', linestyle='-')
    plt.tight_layout()
    fig.autofmt_xdate()
    plt.show()
    
    

In [ ]:
list_of_datetimes, values_exposure_time, values_time_range, values_mcp_gain, values_info, values_trig_level, list_h5_files = zip(*dataset)
list_values_slit_width, list_values_shutter, list_values_mode = filter_data(values_info)
dataset_final = filter_data_property(dataset, list_values_shutter, list_values_mode, list_values_slit_width)
dataset_shortlisted = filter_data_shortlist(dataset_final)
list_of_datetimes_temp_shortlist, values_exposure_time_temp_shortlist, values_time_range_temp_shortlist, values_mcp_gain_temp_shortlist, values_trig_level_temp_shortlist, list_h5_files_temp_shortlist, list_values_slit_width_temp_shortlist = zip(*dataset_shortlisted)

In [ ]:
timestamp_values = []
for val in list_of_datetimes_temp_shortlist:
    timestamp_values.append(val.timestamp())
np.savetxt('/eos/user/a/amthakur/PNGFiles/Streak_20180526_Good_Data/'+'timestamp_values.txt', timestamp_values, fmt='%s')

In [ ]:
import datetime

timestamp_values = np.loadtxt('/eos/user/a/amthakur/PNGFiles/Streak_20180526_Good_Data/'+'timestamp_values.txt')
datetime_values = []
for timestamp in timestamp_values:
    datetime_values.append(datetime.datetime.fromtimestamp(timestamp))

In [ ]:
#write_values(dataset_shortlisted)
list_time_values = []
list_of_files = os.listdir(dir_file_path)
for i in range(0, len(list_of_files)):
    if(i%50==0):
        print(i)
    list_time_values.append(np.loadtxt(dir_file_path+list_of_files[i]))

In [ ]:
# fast_forward_transform_values = performFFT(list_of_files)

# plot_image(datetime_values, 'Fast Fourier Transform Streak Image 2018/05', 'Dates', 'FFT(Real)', np.real(fast_forward_transform_values))
# plot_image(datetime_values, 'Fast Fourier Transform Streak Image 2018/05', 'Dates', 'FFT(Imag)', np.imag(fast_forward_transform_values))

In [ ]:
list_of_files = os.listdir('/eos/user/a/amthakur/PNGFiles/Streak_20180526_Good_Data/')
ImageData = np.loadtxt('/eos/user/a/amthakur/PNGFiles/Streak_20180526_Good_Data/'+list_of_files[0])
fig = plt.figure(figsize=(ImageWidth/100, ImageHeight/100))
ax = fig.add_subplot(111)
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)
ax.set_frame_on(False)
plt.imshow(ImageData)
plt.show()

In [ ]:
from scipy import signal
from numpy.fft import fft

%matplotlib notebook
cm = custom_cmap()
ImageHeight = 512
ImageWidth = 672
pix_per_mm = 43.2

all_max_frequency = []

for i in range(0, len(list_time_values)):
    if(i%10==0):
        print(i)
    axis = list_time_values[i]
    stk_ax1 = (ImageWidth/pix_per_mm)*np.linspace(0,1,ImageWidth)
    stk_ax1 = stk_ax1 - np.mean(stk_ax1)

    dax = axis[-2] - axis[-3]
    axis[-1] = axis[-2]+dax
    axis = axis[0:ImageHeight]

    ImageData = np.loadtxt('/eos/user/a/amthakur/PNGFiles/Streak_20180526_Good_Data/'+list_of_files[i])
#     fig = plt.figure(figsize=(ImageWidth/100, ImageHeight/100))
#     ax = fig.add_subplot(111)
#     ax.get_xaxis().set_visible(False)
#     ax.get_yaxis().set_visible(False)
#     ax.set_frame_on(False)
    filtered_image = signal.medfilt2d(ImageData.astype(float))
    projection = np.mean(ImageData,axis=1)
    zero_offset_proj = projection - np.mean(projection)
    #plt.imshow(ImageData, map=cm)
    #plt.imshow(filtered_image, cmap=cm)
    #plt.plot(np.diff(axis))
    #my_fft = np.abs(fft(projection))
    my_fft = np.abs(fft(zero_offset_proj))
    time_axis = axis
    time_window = time_axis[-1] - time_axis[0]
    delta_t = time_window/len(time_axis)

    maximum_frequency = 1/(delta_t*1E-12)
    minimum_frequency = 1/(time_window*1E-12)

    frequency_axis = (minimum_frequency/1e9)*np.arange(len(time_axis))
    max_amplitude = np.max(my_fft[:254])
    max_frequency = frequency_axis[np.argmax(my_fft[:254])]
    all_max_frequency.append(max_frequency)
    plt.plot(frequency_axis[:254],my_fft[:254],max_frequency,max_amplitude,'ro')

In [ ]:
plot_image(datetime_values, 'Fast Fourier Transform Streak Image 2018/05', 'Dates', 'FFT Max Frequency', all_max_frequency)

In [ ]:
count=0
same_distribution = []
for i in range(0, len(all_max_frequency)):
    if (all_max_frequency[i]>=110):
        same_distribution.append(i)
        count=count+1
print(count)
print(same_distribution)
np.savetxt('/eos/user/a/amthakur/PNGFiles/Streak_20180526_Good_Data/same_distribution.txt', same_distribution)
#123.45480737828495, 118.70654555604322, 113.9582837338015 

In [ ]:
from numpy.fft import fft

same_distribution = np.loadtxt('/eos/user/a/amthakur/PNGFiles/Streak_20180526_Good_Data/same_distribution.txt')
ImageHeight = 512
ImageWidth = 672
pix_per_mm = 43.2
for i in range(0, len(list_of_files)):
    if(i in same_distribution):
        axis = list_time_values[i]
        stk_ax1 = (ImageWidth/pix_per_mm)*np.linspace(0,1,ImageWidth)
        stk_ax1 = stk_ax1 - np.mean(stk_ax1)

        dax = axis[-2] - axis[-3]
        axis[-1] = axis[-2]+dax
        axis = axis[0:ImageHeight]

        ImageData = np.loadtxt('/eos/user/a/amthakur/PNGFiles/Streak_20180526_Good_Data/'+list_of_files[i])
        my_ffts = np.zeros((ImageHeight,ImageWidth))
        max_my_ffts = []
        for col in range(ImageWidth):
            #my_ffts[:,col] = np.abs(fft(my_col))
            my_col = ImageData[:,col]
            max_my_ffts.append(np.float(max(np.abs(fft(my_col)))))
        values = np.cov(max_my_ffts)
        print(i, list_of_files[i], values)
        np.savetxt('/eos/user/a/amthakur/PNGFiles/Streak_20180526_Good_Data_Covariance/'+list_of_files[i], max_my_ffts)